# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 7 2023 11:34AM,258824,10,SONSB0002127,"Nextsource Biotechnology, LLC",Released
1,Mar 7 2023 11:34AM,258824,10,SOTAH0000927,"Nextsource Biotechnology, LLC",Released
2,Mar 7 2023 11:17AM,258823,12,U1235,"Uniderm USA, Inc",Released
3,Mar 7 2023 11:17AM,258823,12,U1236,"Uniderm USA, Inc",Released
4,Mar 7 2023 11:17AM,258823,12,U1237,"Uniderm USA, Inc",Released
5,Mar 7 2023 11:17AM,258823,12,U1238,"Uniderm USA, Inc",Released
6,Mar 7 2023 10:51AM,258820,12,7674629,"Uniderm USA, Inc - Collagenil Line",Released
7,Mar 7 2023 10:46AM,258817,10,MSP219208,"Methapharm, Inc.",Released
8,Mar 7 2023 10:46AM,258817,10,MSP219210,"Methapharm, Inc.",Released
9,Mar 7 2023 10:46AM,258817,10,MSP219187,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,258817,Released,22
21,258818,Released,3
22,258820,Released,1
23,258823,Released,4
24,258824,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258817,NaN,NaN,22.0
258818,NaN,NaN,3.0
258820,NaN,NaN,1.0
258823,NaN,NaN,4.0
258824,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258710,0.0,17.0,28.0
258750,0.0,1.0,0.0
258764,0.0,1.0,0.0
258777,2.0,3.0,10.0
258781,0.0,0.0,14.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258710,0,17,28
258750,0,1,0
258764,0,1,0
258777,2,3,10
258781,0,0,14


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258710,0,17,28
1,258750,0,1,0
2,258764,0,1,0
3,258777,2,3,10
4,258781,0,0,14


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258710,,17,28
1,258750,,1,
2,258764,,1,
3,258777,2,3,10
4,258781,,,14


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 7 2023 11:34AM,258824,10,"Nextsource Biotechnology, LLC"
2,Mar 7 2023 11:17AM,258823,12,"Uniderm USA, Inc"
6,Mar 7 2023 10:51AM,258820,12,"Uniderm USA, Inc - Collagenil Line"
7,Mar 7 2023 10:46AM,258817,10,"Methapharm, Inc."
29,Mar 7 2023 10:44AM,258818,10,"Methapharm, Inc."
32,Mar 7 2023 10:43AM,258816,12,"Uniderm USA, Inc"
33,Mar 7 2023 10:41AM,258815,19,"Methapharm, Inc."
34,Mar 7 2023 10:37AM,258814,19,ACG North America LLC
35,Mar 7 2023 10:33AM,258813,19,"Avet Pharmaceuticals Labs, Inc"
36,Mar 7 2023 10:27AM,258811,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 7 2023 11:34AM,258824,10,"Nextsource Biotechnology, LLC",,,2
1,Mar 7 2023 11:17AM,258823,12,"Uniderm USA, Inc",,,4
2,Mar 7 2023 10:51AM,258820,12,"Uniderm USA, Inc - Collagenil Line",,,1
3,Mar 7 2023 10:46AM,258817,10,"Methapharm, Inc.",,,22
4,Mar 7 2023 10:44AM,258818,10,"Methapharm, Inc.",,,3
5,Mar 7 2023 10:43AM,258816,12,"Uniderm USA, Inc",,,1
6,Mar 7 2023 10:41AM,258815,19,"Methapharm, Inc.",,,1
7,Mar 7 2023 10:37AM,258814,19,ACG North America LLC,,,1
8,Mar 7 2023 10:33AM,258813,19,"Avet Pharmaceuticals Labs, Inc",,,1
9,Mar 7 2023 10:27AM,258811,10,Eywa Pharma Inc.,,,15


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 7 2023 11:34AM,258824,10,"Nextsource Biotechnology, LLC",2,,
1,Mar 7 2023 11:17AM,258823,12,"Uniderm USA, Inc",4,,
2,Mar 7 2023 10:51AM,258820,12,"Uniderm USA, Inc - Collagenil Line",1,,
3,Mar 7 2023 10:46AM,258817,10,"Methapharm, Inc.",22,,
4,Mar 7 2023 10:44AM,258818,10,"Methapharm, Inc.",3,,
5,Mar 7 2023 10:43AM,258816,12,"Uniderm USA, Inc",1,,
6,Mar 7 2023 10:41AM,258815,19,"Methapharm, Inc.",1,,
7,Mar 7 2023 10:37AM,258814,19,ACG North America LLC,1,,
8,Mar 7 2023 10:33AM,258813,19,"Avet Pharmaceuticals Labs, Inc",1,,
9,Mar 7 2023 10:27AM,258811,10,Eywa Pharma Inc.,15,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 7 2023 11:34AM,258824,10,"Nextsource Biotechnology, LLC",2,,
1,Mar 7 2023 11:17AM,258823,12,"Uniderm USA, Inc",4,,
2,Mar 7 2023 10:51AM,258820,12,"Uniderm USA, Inc - Collagenil Line",1,,
3,Mar 7 2023 10:46AM,258817,10,"Methapharm, Inc.",22,,
4,Mar 7 2023 10:44AM,258818,10,"Methapharm, Inc.",3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 7 2023 11:34AM,258824,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN
1,Mar 7 2023 11:17AM,258823,12,"Uniderm USA, Inc",4.0,NaN,NaN
2,Mar 7 2023 10:51AM,258820,12,"Uniderm USA, Inc - Collagenil Line",1.0,NaN,NaN
3,Mar 7 2023 10:46AM,258817,10,"Methapharm, Inc.",22.0,NaN,NaN
4,Mar 7 2023 10:44AM,258818,10,"Methapharm, Inc.",3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 7 2023 11:34AM,258824,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0
1,Mar 7 2023 11:17AM,258823,12,"Uniderm USA, Inc",4.0,0.0,0.0
2,Mar 7 2023 10:51AM,258820,12,"Uniderm USA, Inc - Collagenil Line",1.0,0.0,0.0
3,Mar 7 2023 10:46AM,258817,10,"Methapharm, Inc.",22.0,0.0,0.0
4,Mar 7 2023 10:44AM,258818,10,"Methapharm, Inc.",3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2070451,61.0,4.0,2.0
12,1035268,6.0,1.0,0.0
15,776255,42.0,18.0,0.0
16,258750,0.0,1.0,0.0
19,1035228,12.0,12.0,0.0
20,258807,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2070451,61.0,4.0,2.0
1,12,1035268,6.0,1.0,0.0
2,15,776255,42.0,18.0,0.0
3,16,258750,0.0,1.0,0.0
4,19,1035228,12.0,12.0,0.0
5,20,258807,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,61.0,4.0,2.0
1,12,6.0,1.0,0.0
2,15,42.0,18.0,0.0
3,16,0.0,1.0,0.0
4,19,12.0,12.0,0.0
5,20,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,61.0
1,12,Released,6.0
2,15,Released,42.0
3,16,Released,0.0
4,19,Released,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0
Executing,4.0,1.0,18.0,1.0,12.0,0.0
Released,61.0,6.0,42.0,0.0,12.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,1.0,18.0,1.0,12.0,0.0
2,Released,61.0,6.0,42.0,0.0,12.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,1.0,18.0,1.0,12.0,0.0
2,Released,61.0,6.0,42.0,0.0,12.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()